In [1]:
import datasets

In [2]:
# load dataset
all_data = datasets.load_dataset('wikisql')

Using custom data configuration default
Reusing dataset wikisql (/Users/alexwang/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
#all_data['train'][0]['table']
all_data.keys()

dict_keys(['test', 'validation', 'train'])

In [13]:
# NB: model must be cased
# NB: position of question vs table

def format_input(example, input_token="<i>", col_tok="<c>", typ_tok="<t>"):
    # input_token: string for separating table from question
    question = example['question']
    
    table = example['table']
    table_name = table['name'] if table['name'] else table['id']
    cols = table['header']
    types = table['types']
    linearized_table = f'{table_name} '
    for col, typ in zip(cols, types):
        linearized_table += f'{col_tok} {col} {typ_tok} {typ} '
    
    return f"{linearized_table} {input_token} {question}"
    
    
def format_output(example):
    target = example['sql']['human_readable']
    
    table = example['table']
    table_name = table['name'] if table['name'] else table['id']
    target = target.replace('FROM table', f'FROM {table_name}')
    return target
    
    
def format_example(example, sep_token="<s>", input_token="<i>",
                   col_tok="<c>", typ_tok="<t>"):
    example_input = format_input(example, input_token=input_token)
    example_output = format_output(example)
    return f"{example_input} {sep_token} {example_output}"


def process_dataset(data, max_examples=-1,
                    example_separator="--SEPARATOR--"):
    """
    """
    n_exs = 0
    processed_examples = []
    for example in data:
        processed_examples.append(format_example(example))
        n_exs += 1
        if max_examples > 0 and n_exs >= max_examples:
            break
            
    print(f"\tProcessed {len(processed_examples)} examples")
    return f"\n{example_separator}\n".join(processed_examples)

In [16]:
SPLITS = ["train", "validation", "test"]
max_examples=10000

for split in SPLITS:
    split_data = all_data[split]
    print(f"Processing {split}")
    processed_data = process_dataset(split_data, max_examples=max_examples)
    with open(f'data/{split}.nexs{max_examples}.txt', 'w') as out_fh:
        out_fh.write(processed_data)

Processing train
	Processed 10000 examples
Processing validation
	Processed 8421 examples
Processing test
	Processed 10000 examples
